# Logging models with MLflow

The following notebook explains the differences between an asset and a model in MLflow and how to transition from one to the other. It also contains working examples to start logging models as MLflow models depending on the characteristics of the assets you are dealing with and how they are supported on MLflow.

### Prerequisites to run this notebook

In [ ]:
# Ensure you have the dependencies for this notebook
%pip install -r logging_model_with_mlflow.txt

### About the examplem

To show the different strategies, let's consider the [Heart Disease Data Set](https://archive.ics.uci.edu/ml/datasets/heart+disease). To simplify the modeling part, we are going to use only a XGBoost based model and explore different ways to get the model logged as needed.

In [1]:
import mlflow

mlflow.set_experiment("heart-disease-classifier")

<Experiment: artifact_location='', experiment_id='9fad101d-5b6c-4503-8564-a3335d1f2101', lifecycle_stage='active', name='heart-disease-classifier', tags={}>

Reading the data:

In [2]:
import pandas as pd

file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
df = pd.read_csv(file_url)
df["thal"] = df["thal"].astype("category").cat.codes

Let's do some basic train-test split:

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1), df["target"], test_size=0.3
)

## What's the difference between an artifact and a model?

### Artifacts

Any file generated (and captured) from an experiment's run or job is an artifact. It may represent a model serialized as a Pickle file, the weights of a PyTorch or TensorFlow model, or even a text file containing the coefficients of a linear regression. Other artifacts can have nothing to do with the model itself, but they can contain configuration to run the model, pre-processing information, sample data, etc. 

As you can see, an artifact can come in any format. You can log artifacts in MLflow in the same way you log a file with Azure ML SDK v1 (`Run.log_file`). 

```python
mlflow.log_artifact()
```

### Logging and artifact

Let's see first how you log an artifact so we can later compare with logging and loading models. The following sample logs a model trained with XGBoost as an artifact.

In [9]:
import mlflow
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

with mlflow.start_run():
    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_pred = model.predict(X_test)

    # Saving the model to the file system in UBJ (binary) format.
    model.save_model("model.ubj")

    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_artifact("./model.ubj")

If we explore the artifacts in the run now:

In [13]:
last_run_id = mlflow.last_active_run().info.run_id
client = mlflow.tracking.MlflowClient()
client.list_artifacts(last_run_id)

[<FileInfo: file_size=-1, is_dir=False, path='model.ubj'>]

To load this model back, we need to download it locally and then load it using XGBoost directly:

In [15]:
model_path = client.download_artifacts(run_id=last_run_id, path="model.ubj")

In [19]:
model = XGBClassifier()
model.load_model(fname=model_path)
model

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

### Models

On the other hand, a model in MLflow is also an artifact, as it matches the definition we just made about them above. However, we make stronger assumptions about this type of artifacts. Such assumptions allow us to create a clear contract between the saved artifacts and what they mean. When you log your models as artifacts (simple files), you need to know what the model builder meant for each of them in order to know how to load it. When you log your models as a Model entity, you should be able to tell what it is based on the contract we just mentioned. 

MLflow adopts the MLModel format as a way to create this contract between the artifacts and what they represent. The MLModel format stores assets in a folder, where there is a particular file named `MLModel`. This file is the single source of truth about how a model can be loaded and used.

![](https://miro.medium.com/max/325/1*SASE6235jPLuzqrHtj_Fzg.png)

Considering the variety of machine learning frameworks available to use, MLflow introduced the concept of flavor which indicates what to expect for a given model created with a given framework. For instance, TensorFlow has its own flavor on MLflow which specifies how a TensorFlow model should be persisted and loaded. Because each model flavor indicates how they want to persist and load models, the MLModel format doesn't enforce a single standard that all the models need to support. Such decision allows each flavor to use the methods that provide the best performance or best support according to their best practices, without compromising compatibility with the standard.

## How do I start logging models instead of artifacts?

If you are working on a new project we encourage to start to embrace the MLflow MLModel format to log your models instead of relying on logging artifacts. Models in the MLFlow format enjoy of a better experience in Azure ML like no-code deployments on our managed inference services.

### Logging models using `autolog()`

One of the simplest ways to start using this approach is by using MLflow autolog functionality. Autolog allows MLflow to instruct the framework associated to with the framework you are using to log all the metrics, parameters, artifacts and models that the framework considers relevant. By default, most models will be log if autolog is enabled. Some flavors may decide not to do that in specific situations. For instance, the flavor PySpark won't log models if they exceed a certain size. 

You can turn on autologging by using the autolog method for the framework you are using. For instance, for XGBoost models,

```python
mlflow.xgboost.autolog()
```

Our training would look like this:

In [ ]:
import mlflow
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

with mlflow.start_run():

    mlflow.xgboost.autolog()

    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

Some times, autolog can't guess the best way to log a model and hence you would need to do it yourself. For those cases, you can indicate autolog to log everything but the model. You do that using the parameter `log_models=False` of `autolog()` function.

### Logging models supported by MLFlow

If you need to log the models in a particular way, then you can use the method `log_model` to log the models as you need to. Usually, you will log the model in this way when:

* You need to indicate `pip` packages or dependencies different from the ones that are automatically detected.
* You need to indicate a `conda` environment different from the default one.
* Your models uses a signature different from the one inferred. This is specifically important when you deal with inputs that are tensors where the signature needs specific shapes.
* You want to include input examples.
* You want to include specific artifacts into the package that will be needed.
* Somehow the default behaviour of autolog doesn't fill your needs.

To log a model, you use the `log_method` model of the flavor you are working with. For instance, the following code logs a model for an XGBoost classifier:

In [ ]:
import mlflow
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from mlflow.models import infer_signature

with mlflow.start_run():
    mlflow.xgboost.autolog(log_models=False)

    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    signature = infer_signature(X_test, y_test)
    mlflow.xgboost.log_model(model, "classifier", signature=signature)

If you need to indicate a custom environment with packages, you can use:

```python
from mlflow.utils.environment import _mlflow_conda_env

custom_packages =_mlflow_conda_env(
    additional_conda_deps=None,
    additional_pip_deps=["xgboost==1.5.2"],
    additional_conda_channels=None,
)

mlflow.xgboost.log_model(model, "classifier", conda_env=custom_packages, signature=signature)
```

### Logging custom models

Mlflow supports a big variety of frameworks, including FastAI, MXNet Gluon, PyTorch, TensorFlow, XGBoost, CatBoost, h2o, Keras, LightGBM, MLeap, ONNX, Prophet, spaCy, Spark MLLib, Scikit-Learn, and Statsmodels. However, they may be times where you need to change how a flavor works or even log a model that uses custom objects that are not part of any particular framework. For those cases, you may need to create a custom model flavor.

Other typical cases where you need to create a custom model is when you need to extend beyond the `predict` function available in the framework you are using. For instance, it is typical that people implement forecasting routines using Scikit-learn. However, Scikit-learn doesn't have a `forecast` function. If you want to enable this `forecast` function in your model, you may need to create a custom model flavor.

#### Logging custom models that are serializable

Python objects are serializable when the object can be stored in the file system as a file. During runtime, the object can be materialized from such file and all the values, properties and methods available when it was saved will be restored.

> **Note:** If your model implements the Scikit-learn API, then you can use the Scikit-learn flavor to log the model. For instance, if your model used to (or can be) persisted in Pickle format and the object has methods `predict` and `predict_proba` (at least), then you can use `mlflow.sklearn.log_model` to log it inside a MLflow run.

For this type of models, MLflow introduces a flavor called `pyfunc` (standing from Python function). Basically this flavor allows you to log any object you want as a model, as log as it satisfies two conditions:

- The Python objects inherits from `mlflow.pyfunc.PythonModel`.
- You implement the method `predict` (at least).



The following sample wraps a model created with XGBoost to make it behaves in a different way to the default implementation of the XGBoost flavor (it returns the probabilities instead of the classes):

In [ ]:
from mlflow.pyfunc import PythonModel, PythonModelContext


class ModelWrapper(PythonModel):
    def __init__(self, model):
        self._model = model

    def predict(self, context: PythonModelContext, data):
        # You don't have to keep the semantic meaning of `predict`. You can use here model.recommend(), model.forecast(), etc
        return self._model.predict_proba(data)

    # You can even add extra functions if you need to. Since the model is serialized,
    # all of them will be available when you load your model back.
    def predict_batch(self, data):
        pass

Then, a custom model can be logged in the run like this:

In [ ]:
import mlflow
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from mlflow.models import infer_signature

with mlflow.start_run():
    mlflow.xgboost.autolog(log_models=False)

    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_probs = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_probs.argmax(axis=1))
    mlflow.log_metric("accuracy", accuracy)

    signature = infer_signature(X_test, y_probs)
    mlflow.pyfunc.log_model(
        "classifier", python_model=ModelWrapper(model), signature=signature
    )

> Important: note how the `infer_signature` method now uses `y_probs` to infer the signature. Our target column has the target class, but our model now returns the 2 probabilities for each class.

#### Logging custom models that are not serializable

Models that are not serializable means that they cannot be serialized as a Pickle file. This includes models that holds references to code that can't be serialized, that do not support serialization, or that provides a more efficient way to be persisted in disk.

In this case, you are required to use a different method to persist the artifacts that you need for your model to run. Then, Mlflow will snapshot all these artifacts and package them all for you. You have two different ways to do this, depending on your preferences:

##### Option 1: Use artifacts with the `PythonModel` object

Use this if you want to retain the state of your model's properties. For instance, in a recommender system you might want to store the number of elements to recommend to any user as a parameter. Here, you will implement a model wrapper as you did in the option above, but in this case you will use `artifacts` to indicate MLflow extra files that you want to include for loading the model state.

To log a custom model using artifacts, you can do something as follows:


```python
model_path = 'xgb.model'
model.save_model(model_path)

mlflow.pyfunc.log_model("classifier", 
                        python_model=ModelWrapper(),
                        artifacts={ 'model': model_path },
                        signature=signature)
```

A few things to notice:

* The model was saved using the `save` method of the framework used (it's not saved as a pickle).
* `ModelWrapper()` is the model wrapper, but the model is not passed as a parameter to the constructor.
* A new parameter is indicated, `artifacts`, that is a dictionary of `key`, `path` where `key` is the name of the artifact, and `path` is the local path in the file system where the artifact is stored.

The corresponding model wrapper then would look as follows:

In [ ]:
from mlflow.pyfunc import PythonModel, PythonModelContext


class ModelWrapper(PythonModel):
    def load_context(self, context: PythonModelContext):
        from xgboost import XGBClassifier

        self._model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
        model.load_model(context.artifacts["model"])

    def predict(self, context: PythonModelContext, data):
        return self._model.predict_proba(data)

The complete training routine would look as follows:

In [ ]:
import mlflow
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from mlflow.models import infer_signature

with mlflow.start_run():
    mlflow.xgboost.autolog(log_models=False)

    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_probs = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_probs.argmax(axis=1))
    mlflow.log_metric("accuracy", accuracy)

    model_path = "xgb.model"
    model.save_model(model_path)

    signature = infer_signature(X_test, y_probs)
    mlflow.pyfunc.log_model(
        "classifier",
        python_model=ModelWrapper(),
        artifacts={"model": model_path},
        signature=signature,
    )

##### Option 2: Use a loader module

Sometimes your model logic is complex and there are several source code files being used to make your model work. This would be the case when you have a Python library for your model for instance. In this scenario, you want to package the library all along with your model so it can move from one place to another as a single piece.

MLflow supports this kind of models too by allowing you to specify any arbitrary source code to package along with the model.

```python
model_path = 'xgb.model'
model.save_model(model_path)

mlflow.pyfunc.log_model("classifier", 
                        data_path=model_path,
                        code_path=['loader_module.py'],
                        loader_module='loader_module'
                        signature=signature)
```

A few things to notice:

* The model was saved using the `save` method of the framework used (it's not saved as a pickle).
* A new parameter, `data_path`, was added pointing to the folder where the model's artifacts are located. This can be a folder or a file. Whatever is on that folder or file, it will be packaged with the model.
* A new parameter, `code_path`, was added pointing to the location where the source code is placed. This can be a path or a single file. Whatever is on that folder or file, it will be packaged with the model.
* The parameter `loader_module` represents a namespace in Python where the code to load the model is placed. You are required to to implement in this namespace a function called `_load_pyfunc(data_path: str)` that received the path of the artifacts (being the value you just passed at `data_path`) and returns an object with a method `predict` (at least).

The corresponding `loader_module.py` implementation would be:

In [ ]:
%%writefile loader_module.py

class MyModel():
    def __init__(self, model):
        self._model = model
        
    def predict(self, data):
        return self._model.predict_proba(data)

def _load_pyfunc(data_path: str):
    import os
    
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.load_model(os.path.abspath(data_path))
    
    return MyModel(model)

Note here that:
    
* The model `MyModel` doesn't inherits from `PythonModel` as we did before, but it has a `predict` function.
* The model's source code is on a file. This can be any source code you want. If your project has a folder `src`, it is a great candidate.
* We added a function `_load_pyfunc` which returns an instance of the model class.

The complete training code would look as follows:

In [ ]:
import mlflow
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from mlflow.models import infer_signature

with mlflow.start_run():
    mlflow.xgboost.autolog(log_models=False)

    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    y_probs = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_probs.argmax(axis=1))
    mlflow.log_metric("accuracy", accuracy)

    model_path = "xgb.model"
    model.save_model(model_path)

    signature = infer_signature(X_test, y_probs)
    mlflow.pyfunc.log_model(
        "classifier",
        data_path=model_path,
        code_path=["loader_module.py"],
        loader_module="loader_module",
        signature=signature,
    )